In [173]:
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

#from lime import lime_text
#from lime.lime_text import LimeTextExplainer

import textwrap
import spacy

import pickle

import lftk


In [174]:
#Modele
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
#from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [161]:
import importlib
import myfunctions

importlib.reload(myfunctions)

<module 'myfunctions' from '/Users/andrzejrostkowski/python-code/PRACA_LICENCJACKA/FINAL/myfunctions.py'>

In [162]:
from myfunctions import evaluate_model, tfidf_vectorize, spacy_embedding, extract_features

In [284]:

# Definiuj listę modeli, które chcesz wczytać
models_tfidf = [
    'Logistic Regression',
    'K-Nearest Neighbors',
    'Random Forest',
    'Multinomial Naive Bayes',
    'SVM',
    'MLP',
]

models_spacy = [
    'Logistic Regression',
    'K-Nearest Neighbors',
    'Random Forest',
    'SVM',
    'MLP',
]

models_linguistic = [
    'Logistic Regression',
    'K-Nearest Neighbors',
    'Random Forest',
    'SVM',
    'MLP',
]


# Stwórz pusty słownik do przechowywania wczytanych modeli
loaded_models_tfidf = {}
loaded_models_spacy = {}
loaded_models_linguistic = {}

for name in models_tfidf:
    # Wczytaj model z pliku
    with open(f'MODELE/{name}_model_tfidf.pkl', 'rb') as f:
        loaded_models_tfidf[name] = pickle.load(f)

for name in models_spacy:
    # Wczytaj model z pliku
    with open(f'MODELE/{name}_model_spacy.pkl', 'rb') as f:
        loaded_models_spacy[name] = pickle.load(f)

for name in models_linguistic:
    # Wczytaj model z pliku
    with open(f'MODELE/{name}_model_linguistic.pkl', 'rb') as f:
        loaded_models_linguistic[name] = pickle.load(f)

In [179]:
loaded_models_tfidf

{'Logistic Regression': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())]),
 'K-Nearest Neighbors': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', KNeighborsClassifier())]),
 'Random Forest': Pipeline(steps=[('tfidf', TfidfVectorizer()),
                 ('clf', RandomForestClassifier())]),
 'Multinomial Naive Bayes': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())]),
 'SVM': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())]),
 'MLP': Pipeline(steps=[('tfidf', TfidfVectorizer()),
                 ('clf', MLPClassifier(max_iter=1000))])}

# Zbiory danych do ewaluacji

In [209]:
data = pd.read_csv('df_preprocessed_v1.csv')

amazon_single = pd.read_csv("DANE/reviews_generated1_cut.csv")
yelp_single = pd.read_csv("DANE/reviews_generated2_cut.csv")
amazon_ext = pd.read_csv("DANE/fake-reviews-dataset.csv")
yelp_ext = pd.read_csv("DANE/combat-ai-restaurants-test.csv")
general_gpt3 = pd.read_csv("DANE/reviews_generated4_cut.csv")
general_gpt4 = pd.read_csv("DANE/reviews_generated5_cut.csv")
llama3 = pd.read_csv("DANE/reviews_generated6_cut.csv")

human = pd.read_excel("DANE/restaurant_reviews_anonymized.xlsx")
# with open("DANE/restaurant_reviews_anonymized.csv", 'rb') as f:
#   human = f.read()


/Users/andrzejrostkowski/python-code/.venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [210]:
for df in [amazon_single,yelp_single,general_gpt3,general_gpt4,llama3]:
    df.drop("Unnamed: 0",axis=1,inplace=True)


In [211]:
amazon_single = amazon_single.rename(columns={"text_": "text"})
amazon_ext = amazon_ext.rename(columns={"text_": "text"})
amazon_ext['target'] = amazon_ext['label'].map({'CG': 1, 'OR': 0})
yelp_ext = yelp_ext.rename(columns={"label": "target"})

human = human.loc[:,['Review','Real']]
human.columns = ["text","target"]
human['target'] = 1 - human['target']

amazon_single = amazon_single[amazon_single.apply(lambda x: len(x['text']) >= 20, axis=1)]

In [305]:
print(yelp_ext)

                                                   text  target
0     I tried the "famous" Mac and Cheese burger at ...       1
1     The Arlo hotel in Soho is great for budget-min...       1
2     I thought the lunch specials at this restauran...       1
3     I had a great experience at Covid this morning...       1
4     Da Toscano is one of the more unique Italian r...       0
...                                                 ...     ...
4758  It says the lunch special is offered every day...       0
4759  I stopped by this restaurant for a quick pre-d...       1
4760  I was excited to try this restaurant after rea...       1
4761  I had a great time at this restaurant! The ser...       1
4762  i've eaten at this restaurant, as well as orde...       0

[4763 rows x 2 columns]


In [225]:
for name,df in zbiory.items():
    print(len(df))


8765
10000
40432
4763
2000
1000
110
1000


In [212]:
zbiory = {"amazon_single": amazon_single, "yelp_single": yelp_single, "amazon_ext": amazon_ext, "yelp_ext": yelp_ext, "general_gpt3": general_gpt3, "general_gpt4": general_gpt4, "human": human, "llama3": llama3}

## TF-IDF

In [218]:
results = pd.DataFrame(columns=['Dataset', 'Model', 'Accuracy TFIDF'])

for name, df in zbiory.items():
    print("=" * 50)  # Dodajemy linie złożoną z 50 znaków "="
    print((f"DATASET {name}..."))
    for model in models_tfidf:
        print(f"Evaluating {model}...")
        metrics = evaluate_model(loaded_models_tfidf[model], df["text"], df["target"], cm=False)
        print(metrics)
        new_row = pd.DataFrame({'Dataset': [name], 'Model': [model], 'Accuracy TFIDF': [metrics[0]]})
        results = pd.concat([results, new_row], ignore_index=True)

DATASET amazon_single...
Evaluating Logistic Regression...
Accuracy:   72.710
Precision:   97.301
Recall:   49.370
F1 score:   65.503
(0.7270964061608671, 0.9730077120822622, 0.49369565217391304, 0.6550331698875108)
Evaluating K-Nearest Neighbors...


/var/folders/8s/bq8p3kcj0n7ckt9bpcm56l8r0000gn/T/ipykernel_25813/1068718450.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)


Accuracy:   76.326
Precision:   83.024
Recall:   69.000
F1 score:   75.365
(0.7632629777524245, 0.8302380329584096, 0.69, 0.7536507182713997)
Evaluating Random Forest...
Accuracy:   69.846
Precision:   94.579
Recall:   45.130
F1 score:   61.104
(0.6984597832287507, 0.9457858769931663, 0.45130434782608697, 0.611037527593819)
Evaluating Multinomial Naive Bayes...
Accuracy:   79.738
Precision:   95.285
Recall:   64.587
F1 score:   76.989
(0.7973759269823161, 0.9528543938422065, 0.6458695652173913, 0.7698885721689557)
Evaluating SVM...
Accuracy:   75.836
Precision:   96.410
Recall:   56.043
F1 score:   70.883
(0.7583571021106674, 0.9640987284966342, 0.5604347826086956, 0.7088259554577949)
Evaluating MLP...
Accuracy:   79.772
Precision:   94.689
Recall:   65.109
F1 score:   77.161
(0.797718197375927, 0.9468858678469807, 0.6510869565217391, 0.7716089140796084)
DATASET yelp_single...
Evaluating Logistic Regression...
Accuracy:   97.970
Precision:   98.036
Recall:   97.816
F1 score:   97.926
(

In [220]:
results

,Dataset,Model,Accuracy TFIDF
0,amazon_single,Logistic Regression,0.727096
1,amazon_single,K-Nearest Neighbors,0.763263
2,amazon_single,Random Forest,0.698460
3,amazon_single,Multinomial Naive Bayes,0.797376
4,amazon_single,SVM,0.758357
5,amazon_single,MLP,0.797718
6,yelp_single,Logistic Regression,0.979700
7,yelp_single,K-Nearest Neighbors,0.841400
8,yelp_single,Random Forest,0.969200
9,yelp_single,Multinomial Naive Bayes,0.973000


## Spacy

In [104]:
# Załaduj model spaCy
nlp = spacy.load("en_core_web_sm")

/Users/andrzejrostkowski/python-code/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [213]:
nlp = spacy.load("en_core_web_sm")

embedded_dfs = {}

for name, df in zbiory.items():
    frame = df['text'].to_frame()
    frame.columns = ['text']
    embedded = spacy_embedding(frame, "text", [], nlp)
    embedded_dfs[f'{name}_embedded'] = embedded


/Users/andrzejrostkowski/python-code/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [215]:
with open('embedded_dfs.pickle', 'wb') as handle:
    pickle.dump(embedded_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [221]:
# Dodajemy nową kolumnę 'F1 Score Spacy' z domyślną wartością NaN
results['Accuracy Spacy'] = np.nan

for name, df in zbiory.items():
    print("=" * 50)  # Dodajemy linie złożoną z 50 znaków "="
    print((f"DATASET {name}..."))
    for model in models_spacy:
        print(f"Evaluating {model}...")
        metrics = evaluate_model(loaded_models_spacy[model], embedded_dfs[f'{name}_embedded'], df["target"], cm=False)
        print(metrics)
        # Dodajemy wyniki do nowej kolumny 'Accuracy Spacy'
        results.loc[(results['Dataset'] == name) & (results['Model'] == model), 'Accuracy Spacy'] = metrics[0]


DATASET amazon_single...
Evaluating Logistic Regression...
Accuracy:   76.497
Precision:   83.669
Recall:   68.609
F1 score:   75.394
(0.7649743297204792, 0.8366914103923648, 0.6860869565217391, 0.7539417104634496)
Evaluating K-Nearest Neighbors...
Accuracy:   69.846
Precision:   79.079
Recall:   57.848
F1 score:   66.817
(0.6984597832287507, 0.7907875185735512, 0.5784782608695652, 0.6681732580037665)
Evaluating Random Forest...
Accuracy:   70.690
Precision:   85.998
Recall:   52.739
F1 score:   65.382
(0.7069024529378208, 0.8599787309464729, 0.5273913043478261, 0.6538202398598572)
Evaluating SVM...
Accuracy:   76.212
Precision:   82.841
Recall:   68.957
F1 score:   75.264
(0.7621220764403879, 0.8284147296944372, 0.6895652173913044, 0.7526396962866295)
Evaluating MLP...
Accuracy:   74.672
Precision:   83.019
Recall:   65.043
F1 score:   72.940
(0.7467199087278951, 0.8301886792452831, 0.6504347826086957, 0.7294002925402243)
DATASET yelp_single...
Evaluating Logistic Regression...
Accura

In [222]:
results

,Dataset,Model,Accuracy TFIDF,Accuracy Spacy
0,amazon_single,Logistic Regression,0.727096,0.764974
1,amazon_single,K-Nearest Neighbors,0.763263,0.698460
2,amazon_single,Random Forest,0.698460,0.706902
3,amazon_single,Multinomial Naive Bayes,0.797376,NaN
4,amazon_single,SVM,0.758357,0.762122
5,amazon_single,MLP,0.797718,0.746720
6,yelp_single,Logistic Regression,0.979700,0.923900
7,yelp_single,K-Nearest Neighbors,0.841400,0.899500
8,yelp_single,Random Forest,0.969200,0.909300
9,yelp_single,Multinomial Naive Bayes,0.973000,NaN


## roBERTa

In [ ]:
# in separate file

## Linguistic

In [123]:
with open('selected_columns.txt', 'r') as f:
    selected_columns = [line.strip() for line in f]

In [124]:
selected_columns

['a_stopword_pw',
 'a_kup_pw',
 'a_subtlex_us_zipf_pw',
 'a_subtlex_us_zipf_ps',
 'simp_adj_var',
 'corr_det_var',
 'bilog_ttr',
 'n_upunct',
 'a_adj_pw',
 'a_adp_pw',
 'a_aux_pw',
 'a_det_pw',
 'a_num_pw',
 'a_punct_pw',
 'a_space_pw',
 'a_det_ps',
 'a_num_ps',
 'a_punct_ps',
 'fogi',
 'cole']

In [148]:
nlp = spacy.load("en_core_web_sm")

/Users/andrzejrostkowski/python-code/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [195]:
print(amazon_single.iloc[2170])
print(amazon_single.isna().sum())
amazon_single = amazon_single.drop(2170)

text      :)
label     CG
target     1
Name: 2170, dtype: object
text      0
label     0
target    0
dtype: int64


In [196]:
amazon_test = amazon_single
amazon_test['length'] = amazon_test['text'].apply(len)
amazon_single_sorted = amazon_test.sort_values('length')
# Zakładam, że 'text' to kolumna, której długość chcemy obliczyć
amazon_single['length'] = amazon_single['text'].apply(len)

# Sortujemy DataFrame według kolumny 'length'
amazon_single_sorted = amazon_single.sort_values('length')

# Wyświetlamy najkrótsze rekordy
print(amazon_single_sorted.head(20))


     text label  target  length
6743    A    OG       0       1
8047    a    OG       0       1
7018    k    OG       0       1
5544    A    OG       0       1
9372    a    OG       0       1
7962   ok    OG       0       2
7406   ok    OG       0       2
7554   :)    OG       0       2
6960   ty    OG       0       2
9612   Ok    OG       0       2
9110   OK    OG       0       2
6961   Ok    OG       0       2
5868   ok    OG       0       2
5073   ok    OG       0       2
7255   ok    OG       0       2
2170   :)    CG       1       2
6835   ok    OG       0       2
6151   ok    OG       0       2
9381   ok    OG       0       2
7743   OK    OG       0       2


In [201]:
# Wybieramy rekordy, które mają mniej niż 100 znaków
short_records = amazon_single.loc[amazon_single['length'] < 20]

# Wyświetlamy te rekordy
print(short_records)

                     text label  target  length
12       Amazing product!    CG       1      16
16           not worth it    CG       1      12
18             Impressed!    CG       1      10
20          Loads of fun!    CG       1      13
24    Highly recommended!    CG       1      19
...                   ...   ...     ...     ...
9950                great    OG       0       5
9960         Nice product    OG       0      12
9965        Works for me.    OG       0      13
9979                   OK    OG       0       2
9986   Works Great Thanks    OG       0      18

[1227 rows x 4 columns]


In [259]:
#losuje po 1000 obserwacji
sampled_dataframes = {}

for name in ['amazon_single','yelp_single', 'amazon_ext', 'yelp_ext', 'general_gpt3']:
    df = zbiory[name]
    sampled_df = df.groupby('target').apply(lambda x: x.sample(n=500)).reset_index(drop=True)
    sampled_dataframes[name] = sampled_df


/var/folders/8s/bq8p3kcj0n7ckt9bpcm56l8r0000gn/T/ipykernel_25813/1280456597.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('target').apply(lambda x: x.sample(n=500)).reset_index(drop=True)
/var/folders/8s/bq8p3kcj0n7ckt9bpcm56l8r0000gn/T/ipykernel_25813/1280456597.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('target').apply(lambda x: x.sample(n=500

In [248]:
for name in sampled_dataframes:
    print(len(sampled_dataframes[name]))
    print(sampled_dataframes[name]['target'].value_counts())


1000
target
0    500
1    500
Name: count, dtype: int64
1000
target
0    500
1    500
Name: count, dtype: int64
1000
target
0    500
1    500
Name: count, dtype: int64
1000
target
0    500
1    500
Name: count, dtype: int64


In [261]:
sampled_dataframes['general_gpt4'] = general_gpt4
sampled_dataframes['human'] = human
sampled_dataframes['llama3'] = llama3

In [262]:
sampled_dataframes.keys()

dict_keys(['amazon_single', 'amazon_ext', 'yelp_ext', 'general_gpt3', 'general_gpt4', 'human', 'llama3'])

In [264]:
# Apply the function to each row in the 'text' column of the DataFrame
from tqdm import tqdm
tqdm.pandas()

dataframes_linguistic = {}

for name, df in sampled_dataframes.items():
    print("=" * 50)  # Dodajemy linie złożoną z 50 znaków "="
    print((f"DATASET {name}..."))
    try:
        data_features = df.join(df['text'].progress_apply(lambda x: extract_features(x, nlp, selected_columns)))
        #data_features = df.join(df['text'].apply(extract_features))
        data_features.to_pickle(f"{name}_linguistic.pkl")
        data_features.to_csv(f"{name}_linguistic.csv")

        dataframes_linguistic[name] = data_features
    except Exception as e:
        print(f"Wystąpił błąd podczas przetwarzania zestawu danych {name}: {e}")

DATASET amazon_single...


100%|██████████| 1000/1000 [16:27<00:00,  1.01it/s]


DATASET amazon_ext...


100%|██████████| 1000/1000 [26:15<00:00,  1.58s/it]


DATASET yelp_ext...


...

100%|██████████| 1000/1000 [38:05<00:00,  2.29s/it]


DATASET human...


100%|██████████| 110/110 [03:09<00:00,  1.72s/it]


DATASET llama3...


100%|██████████| 1000/1000 [34:51<00:00,  2.09s/it]


In [288]:
## zapomnialem o yelp single
tqdm.pandas()
sampled_yelp_single = yelp_single.groupby('target').apply(lambda x: x.sample(n=500)).reset_index(drop=True)
linguistic_yelp_single = sampled_yelp_single.join(sampled_yelp_single['text'].progress_apply(lambda x: extract_features(x, nlp, selected_columns)))
#data_features = df.join(df['text'].apply(extract_features))
linguistic_yelp_single.to_pickle("yelp_single_linguistic.pkl")
linguistic_yelp_single.to_csv("yelp_single_linguistic.csv")


/var/folders/8s/bq8p3kcj0n7ckt9bpcm56l8r0000gn/T/ipykernel_25813/4034006719.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_yelp_single = yelp_single.groupby('target').apply(lambda x: x.sample(n=500)).reset_index(drop=True)
100%|██████████| 1000/1000 [34:01<00:00,  2.04s/it] 


In [295]:
dataframes_linguistic['yelp_single'] = linguistic_yelp_single
sampled_dataframes['yelp_single'] = sampled_yelp_single

In [270]:
for name, df in dataframes_linguistic.items():
    print(name)
    print(dataframes_linguistic[name].columns)

amazon_single
Index(['text', 'label', 'target', 'a_stopword_pw', 'a_kup_pw',
       'a_subtlex_us_zipf_pw', 'a_subtlex_us_zipf_ps', 'simp_adj_var',
       'corr_det_var', 'bilog_ttr', 'n_upunct', 'a_adj_pw', 'a_adp_pw',
       'a_aux_pw', 'a_det_pw', 'a_num_pw', 'a_punct_pw', 'a_space_pw',
       'a_det_ps', 'a_num_ps', 'a_punct_ps', 'fogi', 'cole'],
      dtype='object')
amazon_ext
Index(['category', 'rating', 'label', 'text', 'target', 'a_stopword_pw',
       'a_kup_pw', 'a_subtlex_us_zipf_pw', 'a_subtlex_us_zipf_ps',
       'simp_adj_var', 'corr_det_var', 'bilog_ttr', 'n_upunct', 'a_adj_pw',
       'a_adp_pw', 'a_aux_pw', 'a_det_pw', 'a_num_pw', 'a_punct_pw',
       'a_space_pw', 'a_det_ps', 'a_num_ps', 'a_punct_ps', 'fogi', 'cole'],
      dtype='object')
yelp_ext
Index(['text', 'target', 'a_stopword_pw', 'a_kup_pw', 'a_subtlex_us_zipf_pw',
       'a_subtlex_us_zipf_ps', 'simp_adj_var', 'corr_det_var', 'bilog_ttr',
       'n_upunct', 'a_adj_pw', 'a_adp_pw', 'a_aux_pw', 'a_det_pw', '

In [297]:
for name, df in dataframes_linguistic.items():
    dataframes_linguistic[name] = df.loc[:, selected_columns]

In [272]:
for name, df in dataframes_linguistic.items():
    print(name)
    print(dataframes_linguistic[name].columns)

amazon_single
Index(['a_stopword_pw', 'a_kup_pw', 'a_subtlex_us_zipf_pw',
       'a_subtlex_us_zipf_ps', 'simp_adj_var', 'corr_det_var', 'bilog_ttr',
       'n_upunct', 'a_adj_pw', 'a_adp_pw', 'a_aux_pw', 'a_det_pw', 'a_num_pw',
       'a_punct_pw', 'a_space_pw', 'a_det_ps', 'a_num_ps', 'a_punct_ps',
       'fogi', 'cole'],
      dtype='object')
amazon_ext
Index(['a_stopword_pw', 'a_kup_pw', 'a_subtlex_us_zipf_pw',
       'a_subtlex_us_zipf_ps', 'simp_adj_var', 'corr_det_var', 'bilog_ttr',
       'n_upunct', 'a_adj_pw', 'a_adp_pw', 'a_aux_pw', 'a_det_pw', 'a_num_pw',
       'a_punct_pw', 'a_space_pw', 'a_det_ps', 'a_num_ps', 'a_punct_ps',
       'fogi', 'cole'],
      dtype='object')
yelp_ext
Index(['a_stopword_pw', 'a_kup_pw', 'a_subtlex_us_zipf_pw',
       'a_subtlex_us_zipf_ps', 'simp_adj_var', 'corr_det_var', 'bilog_ttr',
       'n_upunct', 'a_adj_pw', 'a_adp_pw', 'a_aux_pw', 'a_det_pw', 'a_num_pw',
       'a_punct_pw', 'a_space_pw', 'a_det_ps', 'a_num_ps', 'a_punct_ps',
       'fo

In [274]:
sampled_dataframes

{'amazon_single':                                                   text label  target
 0    My second reorder. Have not been disappointed ...    OG       0
 1    Good stuff!  Got my herbs looking green and he...    OG       0
 2    Crazy Cats. well let me tell you it spells onl...    OG       0
 3    This was my first time trying a Larabar. I lov...    OG       0
 4                  Wonderful.  I bought 12 more boxes.    OG       0
 ..                                                 ...   ...     ...
 995  I purchased this cat playpen for my two feline...    CG       1
 996  My kitchen must-have! I sprinkle this organic ...    CG       1
 997  I am amazed by the quality of these Darice B00...    CG       1
 998  I was really excited to get the Fiskars Kangar...    CG       1
 999  My dog smells amazing after using this shampoo...    CG       1
 
 [1000 rows x 3 columns],
 'amazon_ext':                          category  rating label  \
 0                  Kindle_Store_5     5.0    OR 

In [298]:
# Dodajemy nową kolumnę 'F1 Score Spacy' z domyślną wartością NaN
results['Accuracy Linguistic'] = np.nan

for name, df in zbiory.items():
    print("=" * 50)  # Dodajemy linie złożoną z 50 znaków "="
    print((f"DATASET {name}..."))
    for model in models_spacy:
        print(f"Evaluating {model}...")
        metrics = evaluate_model(loaded_models_linguistic[model], dataframes_linguistic[name], sampled_dataframes[name]["target"], cm=False)
        print(metrics)
        # Dodajemy wyniki do nowej kolumny 'Accuracy Spacy'
        results.loc[(results['Dataset'] == name) & (results['Model'] == model), 'Accuracy Linguistic'] = metrics[0]


DATASET amazon_single...
Evaluating Logistic Regression...
Accuracy:   78.700
Precision:   84.915
Recall:   69.800
F1 score:   76.619
(0.787, 0.8491484184914841, 0.698, 0.7661909989023051)
Evaluating K-Nearest Neighbors...
Accuracy:   71.000
Precision:   86.458
Recall:   49.800
F1 score:   63.198
(0.71, 0.8645833333333334, 0.498, 0.631979695431472)
Evaluating Random Forest...
Accuracy:   73.700
Precision:   88.599
Recall:   54.400
F1 score:   67.410
(0.737, 0.8859934853420195, 0.544, 0.6741016109045849)
Evaluating SVM...
Accuracy:   78.700
Precision:   83.450
Recall:   71.600
F1 score:   77.072
(0.787, 0.8344988344988346, 0.716, 0.7707212055974165)
Evaluating MLP...
Accuracy:   74.200
Precision:   89.032
Recall:   55.200
F1 score:   68.148
(0.742, 0.8903225806451613, 0.552, 0.6814814814814815)
DATASET yelp_single...
Evaluating Logistic Regression...
Accuracy:   91.300
Precision:   89.788
Recall:   93.200
F1 score:   91.462
(0.913, 0.8978805394990366, 0.932, 0.914622178606477)
Evaluatin

In [299]:
results

,Dataset,Model,Accuracy TFIDF,Accuracy Spacy,Accuracy Linguistic
0,amazon_single,Logistic Regression,0.727096,0.764974,0.787000
1,amazon_single,K-Nearest Neighbors,0.763263,0.698460,0.710000
2,amazon_single,Random Forest,0.698460,0.706902,0.737000
3,amazon_single,Multinomial Naive Bayes,0.797376,NaN,NaN
4,amazon_single,SVM,0.758357,0.762122,0.787000
5,amazon_single,MLP,0.797718,0.746720,0.742000
6,yelp_single,Logistic Regression,0.979700,0.923900,0.913000
7,yelp_single,K-Nearest Neighbors,0.841400,0.899500,0.848000
8,yelp_single,Random Forest,0.969200,0.909300,0.903000
9,yelp_single,Multinomial Naive Bayes,0.973000,NaN,NaN
